In [2]:
# Importing required libraries
from bs4 import BeautifulSoup
import requests
import csv
import nltk
import pandas as pd
import re
import numpy as np
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet')
nltk.download('words')
nltk.download('omw-1.4')
import string
from nltk.stem import WordNetLemmatizer
import pickle as pk
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
# Reading the data
df=pd.read_csv("BBC News Train.csv",encoding = "ISO-8859-1")
df2=pd.read_csv("BBC News Test.csv",encoding = "ISO-8859-1")
# df=df.drop(['ArticleId'], axis=1)
# df.head()

In [8]:
df=df.drop(['ArticleId'], axis=1)
df.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [9]:
def removePunctuation(tweet):
    for i in string.punctuation:
      tweet=tweet.replace(i,"")
    return tweet

df['Text'] = df['Text'].apply(removePunctuation)
df.head()

,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in 168m payout eighteen former en...,business


In [10]:
def removeStopwords(tweet):
    l=tweet.split()
    s=""
    for i in l:
      if i not in stop_words :
        s=s+i+" "
    return s[:-1]

stop_words = set(stopwords.words("english"))
df['Text'] = df['Text'].apply(removeStopwords)
df.head()

,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens maj...,business
3,lifestyle governs mobile choice faster better ...,tech
4,enron bosses 168m payout eighteen former enron...,business


In [11]:
def tolower(tweet):
    tweet=tweet.lower()
    return tweet

df['Text'] = df['Text'].apply(tolower)
df.head()

,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens maj...,business
3,lifestyle governs mobile choice faster better ...,tech
4,enron bosses 168m payout eighteen former enron...,business


In [12]:
def tokenization(tweet):
  data = word_tokenize(tweet)
  return data

df['Text'] = df['Text'].apply(tokenization)
df.head()

,Text,Category
0,"[worldcom, exboss, launches, defence, lawyers,...",business
1,"[german, business, confidence, slides, german,...",business
2,"[bbc, poll, indicates, economic, gloom, citize...",business
3,"[lifestyle, governs, mobile, choice, faster, b...",tech
4,"[enron, bosses, 168m, payout, eighteen, former...",business


In [13]:
def lemmatize(tweet):

  output = []
  lst = ['a','r','n','v']
  wordNetLemmatizer = WordNetLemmatizer()
  posTag = pos_tag(tweet)
  for word, tag in posTag:
    pos = tag[0].lower()
    if pos not in lst:
      pos = 'n'
    output.append(wordNetLemmatizer.lemmatize(word,pos))
  return output

df['Text'] = df['Text'].apply(lemmatize)
df.head()

,Text,Category
0,"[worldcom, exboss, launch, defence, lawyer, de...",business
1,"[german, business, confidence, slide, german, ...",business
2,"[bbc, poll, indicate, economic, gloom, citizen...",business
3,"[lifestyle, governs, mobile, choice, faster, w...",tech
4,"[enron, boss, 168m, payout, eighteen, former, ...",business
